In [5]:
# %load quants/4_GL_loop.py
import yfinance as yf
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr

yf.pdr_override() # <== that's all it takes :-)
start =dt.datetime(1980,12,1)
now = dt.datetime.now()
stock=""

stock = input("Enter the stock symbol : ") 
while stock != "quit":
  df = pdr.get_data_yahoo(stock, start, now)

  df.drop(df[df["Volume"]<1000].index, inplace=True)

  dfmonth=df.groupby(pd.Grouper(freq="M"))["High"].max()

  glDate=0
  lastGLV=0
  currentDate=""
  curentGLV=0
  for index, value in dfmonth.items():
    if value > curentGLV:
      curentGLV=value
      currentDate=index
      counter=0
    if value < curentGLV:
      counter=counter+1

      if counter==3 and ((index.month != now.month) or (index.year != now.year)):
          if curentGLV != lastGLV:
            print(curentGLV)
          glDate=currentDate
          lastGLV=curentGLV
          counter=0

  if lastGLV==0:
    message=stock+" has not formed a green line yet"
  else:
    message=("Last Green Line: "+str(lastGLV)+" on "+str(glDate))

  print(message)
  stock = input("Enter the stock symbol : ") 




In [6]:
# %load quants/4_GLVexcel.py
import yfinance as yf
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)
start =dt.datetime(1980,12,1)
now = dt.datetime.now()
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from pandas import ExcelWriter
from pandas import ExcelFile
from openpyxl import Workbook
import os

root = Tk()
ftypes = [(".xlsm","*.xlsx",".xls")]
ttl  = "Title"
dir1 = 'C:\\'
#Input a file with symbols in first column
root.fileName = askopenfilename(filetypes = ftypes, initialdir = dir1, title = ttl)
print (root.fileName)

GLlistDATA = pd.DataFrame(columns=['Stock', "Last Greenline", "Date"])

a = pd.read_excel(root.fileName, index_col=0, index_row=0, header=None)
for index, value in a.iterrows():
  stock=index
  df = pdr.get_data_yahoo(stock, start, now)
  df.drop(df[df["Volume"]<1000].index, inplace=True)
  dfmonth=df.groupby(pd.Grouper(freq='M'))['High'].max()
  DOH=""
  GLV=0
  DOHc=""
  GLVc=0
  counter=0
  for index, value in dfmonth.items():
    if value> GLVc:
      GLVc=value
      DOHc=index
      counter=0
    if value < GLVc:
      counter=counter+1
      if counter==3 and ((index.month != now.month) or (index.year != now.year) ):
        GLV=GLVc
        counter=0
        DOH=DOHc
        print(str(DOH))
  GLlistDATA = GLlistDATA.append({'Stock': stock,'Greenline': GLV, 'Date': DOH}, ignore_index=True)
 
print(GLlistDATA)
newFile=os.path.dirname(root.fileName)+"/output.xlsx"
print(newFile)

writer = ExcelWriter(newFile)
GLlistDATA.to_excel(writer,'Sheet1')
writer.save() 
